In [1]:
import rosbag
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob2, re
from collections import Counter
import networkx as nx
import community as comm
%matplotlib inline

# Data filtering and conversion

In [2]:
def TopicFilter(bag):

    topics = bag.get_type_and_topic_info().topics.keys()
    topics_excluing_list = []
    topics_valid_list = []

    for id in xrange(20,40):
        topic_goal = "/epuck_" + str(id) + "/diff_drive_go_to_goal" + "/goal"
        topic_goal_achieved = "/epuck_" + str(id) + "/diff_drive_go_to_goal" + "/goal_achieved"
        topic_distance_to_goal = "/epuck_" + str(id) + "/diff_drive_go_to_goal" + "/distance_to_goal"
        topic_goto = "/epuck_" + str(id) + "/diff_drive_go_to_goal" + "/go_to"
        topic_results = "/epuck_" + str(id) + "/diff_drive_go_to_goal" + "/result"
        topic_cancel = "/epuck_" + str(id) + "/diff_drive_go_to_goal" + "/cancel"
        topic_status = "/epuck_" + str(id) + "/diff_drive_go_to_goal" + "/status"
        topic_cmd_vel = "/epuck_" + str(id) + "/cmd_vel"

        topics_excluing_list.extend([topic_goal, topic_distance_to_goal, topic_goto, topic_goal_achieved, 
                                topic_results, topic_cancel, topic_status, topic_cmd_vel])

        topics_excluing_list.extend(['/rosout', '/rosout_agg'])

    for topic in topics:
        if topic not in topics_excluing_list:
            topics_valid_list.append(topic)
            
    return topics_valid_list

In [3]:
def ROSBagToPandasDataFrame(rosbag_filename):

    bag = rosbag.Bag(rosbag_filename)
    topics_valid_list = TopicFilter(bag)
    msgs = []

    for topic, msg, t in bag.read_messages(topics=topics_valid_list):
        msgs.append({'topic': topic, 'msg': msg, 't': t})
    
    bag.close()
    return pd.DataFrame(msgs)

#  Finishing Time and Success Rate Functions

In [4]:
def MerkleTreeCompletionTimePerRobot(df, robot_topic):
    msgs_topic = df.loc[df['topic'] == robot_topic]['msg']
    msg_begin = msgs_topic.iloc[0]
    msg_end = None
    
    for msg in msgs_topic:        
        if(msg.leafs[-1].completed == True):
            msg_end = msg
            break
            
    return msg_end.header.stamp.secs - msg_begin.header.stamp.secs

In [5]:
def FinishingTimeOneRun(INITIAL_ID, NUMBER_OF_ROBOTS, THRESHOLD, DF):
    robots_finishing_list = []

    for id in xrange(INITIAL_ID, INITIAL_ID+NUMBER_OF_ROBOTS):
        topic_name = '/epuck_' + str(id) + '/comm/merkle_completed'
        try:
            robot_finishing_time = MerkleTreeCompletionTimePerRobot(DF, topic_name)
            if (robot_finishing_time < THRESHOLD):
                robots_finishing_list.append(robot_finishing_time)
        except:
            pass
           
    return max(robots_finishing_list)

In [6]:
def CompletionSuccess(THRESHOLD, FINISHING_TIMES):

    filtered_times = []

    for t in range(0,THRESHOLD):
        filtered_times.append(len([i for i in FINISHING_TIMES if i <= t]))
  
    return np.true_divide(np.array(filtered_times), len(FINISHING_TIMES))

# Communication Cost and Privacy Functions

In [7]:
# function to get unique values 
def unique(list1): 
    x = np.array(list1)
    return np.unique(x)

In [8]:
# get_size function
def get_size(obj, seen=None):
    """Recursively finds size of objects"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size

In [9]:
def sdi(data):
    from math import log as ln
    
    def p(n, N):
        """ Relative abundance """
        if n is  0:
            return 0
        else:
            return (float(n)/N) * ln(float(n)/N)
            
    N = sum(data)
    return -sum(p(n, N) for n in data if n is not 0)

In [10]:
def ExtractCommunicationGraph(INITIAL_ID, NUMBER_OF_ROBOTS, NUMBER_OF_LEAVES, DF):
    G = nx.DiGraph()
    for robotid in xrange(INITIAL_ID, INITIAL_ID+NUMBER_OF_ROBOTS):
        G.add_node(robotid)
        msgs_topic = DF.loc[DF['topic'] == '/epuck_' + str(robotid) + '/comm/merkle_completed']['msg']
        from_robots = []
        for leaf in msgs_topic.iloc[-1].leafs:
            if leaf.received == True:
                from_robots.append(leaf.from_robot)         
        for other_robot in unique(from_robots):
            G.add_edge(other_robot, robotid, weight=Counter(from_robots)[other_robot])
    return G

In [11]:
def CommunicationCost(G, NUMBER_OF_LEAVES, HASH_SIZE_IN_BYTES):  
    number_of_iterations = sum(nx.get_edge_attributes(G,'weight').values())    
    if (number_of_iterations!=0):
        return sum(nx.get_edge_attributes(G,'weight').values()) * (np.log2(NUMBER_OF_LEAVES) + 2) * HASH_SIZE_IN_BYTES

In [16]:
def PrivacyCoefficient(INITIAL_ID, NUMBER_OF_ROBOTS, NUMBER_OF_LEAVES, DF):
    by_all_robots_in_sim = []

    for robotid in xrange(INITIAL_ID, INITIAL_ID+NUMBER_OF_ROBOTS):
        msgs_topic = DF.loc[DF['topic'] == '/epuck_' + str(robotid) + '/comm/merkle_completed']['msg']
        by_this_robot = 0
        for leaf in msgs_topic.iloc[-1].leafs:
            if leaf.received == False and leaf.hash != "":
                by_this_robot =+ 1
        
        by_all_robots_in_sim.append(by_this_robot)

        if sum(by_all_robots_in_sim)==NUMBER_OF_LEAVES:
            return sdi(by_all_robots_in_sim)/np.log(NUMBER_OF_LEAVES)

## Main loop 

In [17]:
path=r'/Users/eddie/experiments_results/5x5-20-robots/data/' # use your path
ALL_FILES = glob2.glob(path + "*.bag")
RUN_THRESHOLD = 21000
NUMBER_OF_ROBOTS = 20
INITIAL_ID = 20
NUMBER_OF_LEAVES = 16
HASH_SIZE_IN_BYTES = 32
LEAF_RANGE = range(16,17)

FINISHING_TIMES = []
COMPLETTION_SUCCESS = []
COMMUNICATION_GRAPHS = []
PRIVACY_COEFFICIENT = []
COMMUNICATION_COST = []

for _file in ALL_FILES:
    DF = ROSBagToPandasDataFrame(_file)
    FINISHING_TIMES.append(FinishingTimeOneRun(INITIAL_ID, NUMBER_OF_ROBOTS, RUN_THRESHOLD, DF))
    COMMUNICATION_GRAPHS.append(ExtractCommunicationGraph(INITIAL_ID, NUMBER_OF_ROBOTS, NUMBER_OF_LEAVES, DF))
    COMMUNICATION_COST.append(CommunicationCost(COMMUNICATION_GRAPHS[-1], NUMBER_OF_LEAVES, HASH_SIZE_IN_BYTES))
    PRIVACY_COEFFICIENT.append(PrivacyCoefficient(INITIAL_ID, NUMBER_OF_ROBOTS, NUMBER_OF_LEAVES, DF))

COMPLETION_SUCCESS = CompletionSuccess(RUN_THRESHOLD, FINISHING_TIMES)

In [18]:
REALROBOT_FINISHING_TIMES = FINISHING_TIMES
REALROBOT_COMMUNICATION_COST = COMMUNICATION_COST
REALROBOT_INFORMATION_DIVERSITY = PRIVACY_COEFFICIENT

%store REALROBOT_FINISHING_TIMES
%store REALROBOT_COMMUNICATION_COST
%store REALROBOT_INFORMATION_DIVERSITY

Stored 'REALROBOT_FINISHING_TIMES' (list)
Stored 'REALROBOT_COMMUNICATION_COST' (list)
Stored 'REALROBOT_INFORMATION_DIVERSITY' (list)


In [19]:
REALROBOT_FINISHING_TIMES

[1881, 2411, 2689, 1693, 1406, 1872, 2322, 1747, 2141, 3022]

In [24]:
ALL_FILES

['/Users/eddie/experiments_results/5x5-16-robots/data/2020-03-21-17-38-08.bag',
 '/Users/eddie/experiments_results/5x5-16-robots/data/2020-03-13-16-06-38.bag',
 '/Users/eddie/experiments_results/5x5-16-robots/data/2020-03-12-03-18-13.bag',
 '/Users/eddie/experiments_results/5x5-16-robots/data/2020-03-15-16-00-38.bag',
 '/Users/eddie/experiments_results/5x5-16-robots/data/2020-03-21-19-03-32.bag',
 '/Users/eddie/experiments_results/5x5-16-robots/data/2020-02-14-22-59-25.bag',
 '/Users/eddie/experiments_results/5x5-16-robots/data/2020-04-23-15-07-21.bag',
 '/Users/eddie/experiments_results/5x5-16-robots/data/2020-02-16-04-14-03.bag',
 '/Users/eddie/experiments_results/5x5-16-robots/data/2020-02-14-02-26-23.bag',
 '/Users/eddie/experiments_results/5x5-16-robots/data/2020-03-12-01-32-10.bag']

In [25]:
df = ROSBagToPandasDataFrame('/Users/eddie/experiments_results/5x5-16-robots/data/2020-04-23-15-07-21.bag')

In [26]:
FinishingTimeOneRun(20, 16, 21000, df)

2154